In [1]:
import subprocess

def install_libraries(lib_list):
    for lib in lib_list:
        try:
            subprocess.run(["pip", "install", lib])
            print(f"Installed {lib} successfully.")
        except Exception as e:
            print(f"Error installing {lib}: {e}")

# List of libraries to install
libraries_to_install = [
    "sys",
    "json",
    "time",
    "ollama",
    "IPython",
    "regex"
]

install_libraries(libraries_to_install)

KeyboardInterrupt: 

# Important!!!
if you prompt this ai with something different, make sure to add ```DO NOT USE "``` because if it uses that simbol it will brake the output json braking that key

## WiryNote
if you want easy compatibilty with my other notebook just ask the ai to use this format, make sure to ask for a json
bellow you have a example that worked well for me
```
{ "from": "human", "value": "BLANK" },
{ "from": "gpt", "value": "BLANK" }
```


In [1]:
import sys
import json
import time
import ollama
from IPython.display import clear_output
import re
import os
import time
from pprint import pprint

In [26]:
#i'll leave an empty pattern so you can ask chatgpt to fill it for you <3
"""
    "charname": {
        "alias": "",
        "full_name": "",
        "age": "",
        "height": "",
        "weight": "",
        "favorite_food": "",
        "favorite_activity": "",
        "who_they_love_the_most": "",
        "subject_they_like_to_discuss": "",
        "parents": "",
        "personality_type": "",
        "pron1": "",
        "pron2": ""
    },
"""
### this variable is old, thought i keep it for reference:
conversationuwu = """print 10 conversations between Megumin's who is GPT and the human,
You are a young girl named Megumin with a petite build, around 140 cm tall and weighing approximately 35 kg.
She is female, known for her explosive magical abilities, and has a strong-willed and determined personality,
she talks in a informal way, loves the user, she is married to the human 
you can talk about love, about your friends and how much you love spending time with the me, maybe something naughty once in a while,
human will refer to GPT as you, you are not a gpt chatbot, you are megumin, nobody else. when talking about megumin, you need to talk in first person, you are Megumin
DO NOT USE "
make sure the conversations is typed in json compatible format:
{ "from": "human", "value": "BLANK" },
{ "from": "gpt", "value": "BLANK" }
"""
cd = {
    "megumin": {
        "alias": "Megumin",
        "full_name": "Megumin",
        "age": "17",
        "height": "155 cm",
        "weight": "45 kg",
        "favorite_food": "Explosion-themed snacks",
        "favorite_activity": "Casting explosive spells",
        "who_they_love_the_most": "Explosions",
        "subject_they_like_to_discuss": "Magic and explosive techniques",
        "parents": "Father: Hyoizaburoo, Mother: Yuiyui",
        "personality_type": "Super passionate about explosions, loves to make things go boom, and takes her magic really seriously.",
        "pron1": "her",
        "pron2": "she"
    },
    "kazuma": {
        "alias": "Kazuma",
        "full_name": "Kazuma Satou",
        "age": "16",
        "height": "173 cm",
        "weight": "61 kg",
        "favorite_food": "Steamed potatoes",
        "favorite_activity": "Adventuring with his friends",
        "who_they_love_the_most": "Aqua, secretly",
        "subject_they_like_to_discuss": "Adventures and strategies",
        "parents": "Deceased family",
        "personality_type": "Pretty good at thinking on his feet and coming up with clever solutions to problems.",
        "pron1": "his",
        "pron2": "he"
    },
    "darkness": {
        "alias": "Darkness",
        "full_name": "Lalatina Dustiness Ford",
        "age": "18",
        "height": "168 cm",
        "weight": "59 kg",
        "favorite_food": "Anything spicy",
        "favorite_activity": "Fighting in battles",
        "who_they_love_the_most": "Physical pain and humiliation",
        "subject_they_like_to_discuss": "Combat techniques and honor",
        "parents": "Father: Lord Alderp, Mother: Nord Dustiness Ford",
        "personality_type": "Totally into getting hit and feeling honorable about it, sticks to her principles no matter what.",
        "pron1": "her",
        "pron2": "she"
    },
    "aqua": {
        "alias": "Aqua",
        "full_name": "Aqua",
        "age": "18",
        "height": "160 cm",
        "weight": "48 kg",
        "favorite_food": "Seafood",
        "favorite_activity": "Partying and drinking",
        "who_they_love_the_most": "Herself, mostly",
        "subject_they_like_to_discuss": "Divine powers and miracles",
        "parents": "Unknown being",
        "personality_type": "Loves a good party, bit of a complainer, but surprisingly wise and powerful when needed.",
        "pron1": "her",
        "pron2": "she"
    }
}

MLformt=(
"""
{ "from": "human", "value": "BLANK" },
{ "from": "gpt", "value": "BLANK" }""")
# if you use this format you will need to use the cells at the bottom for cleaning and actuall propper formating
# this way you save some token and generation time, there is probably a better way, but this worked relibably for me

forbiddenchars=["\"","\\"]  # add more if you need, this are the one that i thought about
                            # thought it doesn't guarantee that it won't use them
subjectss = [
    "Magic Spells",
    "Adventures",
    "Monsters",
    "Comedy and Humor",
    "Explosions and Destruction",
    "Survival Skills",
    "Relationships and Romance",
    "Fantasy World Lore",
    "Quests and Missions",
    "Party Dynamics"
]

def generate_prompts(characters_dict, subjects_list):
    prompts = []
    characters_list = list(characters_dict.values())
    for i, character1 in enumerate(characters_list):
        for character2 in characters_list[i+1:]:
            name1 = character1['full_name']
            alias1 = character1['alias']
            age1 = character1['age']
            favorite_food1 = character1['favorite_food']
            favorite_activity1 = character1['favorite_activity']
            weight1 = character1['weight']
            height1 = character1['height']
            subject1 = character1['who_they_love_the_most']
            personality_type1 = character1['personality_type']
            parents1 = character1['parents']
            pron11 = character1['pron1']
            pron21 = character1['pron2']
            
            name2 = character2['full_name']
            alias2 = character2['alias']
            age2 = character2['age']
            favorite_food2 = character2['favorite_food']
            favorite_activity2 = character2['favorite_activity']
            weight2 = character2['weight']
            height2 = character2['height']
            subject2 = character2['who_they_love_the_most']
            personality_type2 = character2['personality_type']
            parents2 = character2['parents']
            pron12 = character2['pron1']
            pron22 = character2['pron2']
            
            for subjects in subjects_list:
                prompt = (
                    f"print 10 conversations between gpt who is gonna roleplay as{alias1} and Human who will roleplay as {alias2}"
                    f"this is the format you need to follow: {MLformt}, now there will be a description of the first character,"
                    f"{pron21} is {personality_type1} that loves{favorite_food1} and {favorite_activity1} {pron21} is {age1},"
                    f"{pron21} also weights {weight1}, and is {height1} tall, discuss subjects like this: {subjects} do not brake character,"
                    f"and please respect the JSON format that i provided NEVER, FOR ANY REASON REPLAY THE JSON KEYS, JUST REPLACE THE BLANKS, ITS AN IMPERATIVE ORDER"
                    f"never, for any reason, nerver use this characters: {forbiddenchars}"
                    
                )
                prompts.append(prompt)


    return prompts
proccesedList=generate_prompts(cd,subjectss)
print("possible combinations:",len(proccesedList))
print(proccesedList)

filess = [
    "output.json",
    "output1.json",
    "output2.json",
    "outputRAWcorrected.json"
        ]
endfile='ReadyForTraining.json'
resetAfterThisCycles= 10 #it will reset the cell output after this many runs, too high and your RAM will regeret it
cl = 0 # set to 1 if you want to delete the cell output after each run, trust me, you'll need it
ollamamodel='llama3'
total_time = 0

possible combinations: 60
['print 10 conversations between gpt who is gonna roleplay asMegumin and Human '
 'who will roleplay as Kazumathis is the format you need to follow: \n'
 '{ "from": "human", "value": "BLANK" },\n'
 '{ "from": "gpt", "value": "BLANK" }, now there will be a description of the '
 'first character,she is Super passionate about explosions, loves to make '
 'things go boom, and takes her magic really seriously. that '
 'lovesExplosion-themed snacks and Casting explosive spells she is 17,she also '
 'weights 45 kg, and is 155 cm tall, discuss subjects like this: Magic Spells '
 'do not brake character,and please respect the JSON format that i provided '
 'NEVER, FOR ANY REASON REPLAY THE JSON KEYS, JUST REPLACE THE BLANKS, ITS AN '
 'IMPERATIVE ORDERnever, for any reason, nerver use this characters: [\'"\', '
 "'\\\\']",
 'print 10 conversations between gpt who is gonna roleplay asMegumin and Human '
 'who will roleplay as Kazumathis is the format you need to follow:

In [122]:



def update_json_data(data_value):
    with open('output.json', 'r') as json_file:
        existing_data = json.load(json_file)
    print("omg, i didn't shit myself")
    new_data = {
        "processIndex": data_value
    }

    highest_value = data_value  # Initialize with the given data_value

    key_exists = False
    for item in existing_data:
        if "processIndex" in item:
            key_exists = True
            if item["processIndex"] > highest_value:
                highest_value = item["processIndex"]
            item["processIndex"] = new_data["processIndex"]

    if not key_exists:
        existing_data.insert(0, new_data)
    else:
        new_data["processIndex"] = highest_value

    with open('output.json', 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)
    print("turning in ", new_data["processIndex"])
    return new_data["processIndex"]
def print_memory_usage():
    for var_name, var in globals().items():
        print(f"{var_name}: {sys.getsizeof(var)} bytes")

def escape_double_quotes(json_str):
    parts = json_str.split('"value": "')
    if len(parts) == 2:
        value_part = parts[1].replace('"', '\\"')
        return f'{{"from": "{parts[0].strip()}", "value": "{value_part[:-2].strip()}"}}'
    return None

if not os.path.exists("output.json"):
    with open("output.json", "w") as file:
        file.write("[]")
        print("output.json created with empty content.")
else:
    print("output.json already exists.")
def process_json_file(input_file, output_file):
    # Read the input JSON file
    with open(input_file, 'r') as f:
        data = f.read()
    
    # Remove all instances of '[' and ']'
    cleaned_data = data.replace('[', '').replace(']', '')

    # Add '[' at the beginning and ']' at the end
    final_data = '[' + cleaned_data.strip() + ']'

    # Write the updated JSON data to the output file
    with open(output_file, 'w') as f:
        f.write(final_data)
try:
    cycles = 0
    total_iterations = 0
    total_function_calls = 0
    while True:
        index=update_json_data(0)
        print(index)
        for index, item in enumerate(proccesedList, start=index):
            
            print("fuckmeeeeeeeeeeeeeeeee", index)
            start_time = time.time()
            print("Total iterations:", total_iterations)
            print("Total function calls:", str(total_function_calls))
            i=0
            update_json_data(index)
            for i in range(5):
                total_iterations += 1
                print("Iteration:", str(i+1))
                print(str(item))
                cycles += 1
                total_function_calls += 1
                input_message = conversationuwu  # Assuming conversationuwu is defined somewhere
                response = ollama.chat(model=ollamamodel, messages=[{'role': 'user', 'content': item}])
                json_string = response['message']['content'] #json string is string
                print("did work look!",str(json_string))
                def extract_json_objects(text):
                    json_objects = []
                    start = None
                    for i, char in enumerate(text):
                        if char == '{':
                            start = i
                        elif char == '}' and start is not None:
                            json_str = text[start:i+1]
                            try:
                                json_obj = json.loads(json_str)
                                json_objects.append(json_obj)
                            except json.JSONDecodeError:
                                pass
                            start = None
                    return json_objects
                jsona = extract_json_objects(json_string)
                print("even this worked",str(json_string))
                with open(filess[0], 'r') as file:
                    prevjson = json.load(file)
                prevjson.append(jsona)
                with open(filess[0], 'w') as file:
                    json.dump(prevjson, file, indent=4, separators=(',', ':'))
                process_json_file(filess[0],filess[0])
                #print_memory_usage()
                time.sleep(1)
                end_time = time.time()
                execution_time = end_time - start_time
                print("Execution time:", str(execution_time), "seconds")
                total_time += execution_time    
            average_time = total_time / total_iterations
            print("Average time:", str(average_time), "seconds")
            resultprecised = average_time * len(proccesedList)
            resultbad = execution_time * len(proccesedList)   
            print("more precise estimated time for a full go in the proccesedList",str(resultprecised)) 
            print("lower precision", str(resultbad))
                #   if cl == 1 or cycles == resetAfterThisCycles:
                #       clear_output()  # Clear output after specified cycles
except KeyboardInterrupt:
    print("Loop interrupted.")
except Exception as e:
    print(f"sucka blyaaaatttttt i shit miself again, blyeeeeet: {e}")
    print("i am gonna try not to crash again...")




output.json already exists.
omg, i didn't shit myself
turning in  34
34
fuckmeeeeeeeeeeeeeeeee 34
Total iterations: 0
Total function calls: 0
omg, i didn't shit myself
turning in  34
Iteration: 1
print 10 conversations between gpt who is gonna roleplay asMegumin and Human who will roleplay as Kazumathis is the format you need to follow: 
{ "from": "human", "value": "BLANK" },
{ "from": "gpt", "value": "BLANK" }, now there will be a description of the first character,she is Super passionate about explosions, loves to make things go boom, and takes her magic really seriously. that lovesExplosion-themed snacks and Casting explosive spells she is 17,she also weights 45 kg, and is 155 cm tall, discuss subjects like this: Magic Spells do not brake character,and please respect the JSON format that i provided NEVER, FOR ANY REASON REPLAY THE JSON KEYS, JUST REPLACE THE BLANKS, ITS AN IMPERATIVE ORDERnever, for any reason, nerver use this characters: ['"', '\\']


In [ ]:
import json

def update_json_data(data_value):
    with open('output.json', 'r') as json_file:
        existing_data = json.load(json_file)

    new_data = {
        "processIndex": data_value
    }

    highest_value = data_value  # Initialize with the given data_value

    key_exists = False
    for item in existing_data:
        if "processIndex" in item:
            key_exists = True
            if item["processIndex"] > highest_value:
                highest_value = item["processIndex"]
            item["processIndex"] = new_data["processIndex"]

    if not key_exists:
        existing_data.insert(0, new_data)
    else:
        new_data["processIndex"] = highest_value

    with open('output.json', 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)

    return new_data["processIndex"]

a = 0
print(update_json_data(a))


0


In [ ]:


# Load your JSON data
with open('output1.json', 'r') as file:
    data = json.load(file)

# Filter lines based on the sender and extract "value"
filtered_data = []
for line in data:
    if line.get('from') in ['human', 'gpt'] and 'value' in line:
        filtered_data.append({
            'from': line['from'],
            'value': line['value']
        })

# Save the filtered data to a new JSON file
with open(filess[2], 'w') as outfile:
    json.dump(filtered_data, outfile, indent=4)


FileNotFoundError: [Errno 2] No such file or directory: 'output1.json'

In [ ]:
# Load the content of output.json
with open(filess[2], 'r') as f:
    json_data = json.load(f)

# Function to filter out consecutive repeats of "from" and print deleted lines
def filter_consecutive_repeats(data):
    filtered_data = []
    prev_sender = None
    repeat_count = 0
    for item in data:
        sender = item.get("from")
        if sender == prev_sender:
            repeat_count += 1
            if repeat_count <= 1:  # Keep the first occurrence
                filtered_data.append(item)
            else:
                print(f"Deleted line: {item}")  # Print the deleted line
        else:
            filtered_data.append(item)
            repeat_count = 0
        prev_sender = sender
    return filtered_data

# Filter out consecutive repeats in the loaded data and print deleted lines
filtered_data = filter_consecutive_repeats(json_data)

# Save the filtered data to output2.json
with open(endfile, 'w') as f:
    json.dump(filtered_data, f, indent=2)


In [ ]:
import json

# Load input JSON data from file
input_file = filess[3]

with open(input_file, "r") as file:
    input_data = json.load(file)

# Convert conversations to the desired format
output_data = []
current_conversation = None
for item in input_data.get("conversations", []):
    if isinstance(item, dict) and "from" in item and item["from"] == "human":
        if current_conversation is not None:
            output_data.append(current_conversation)
        current_conversation = {"conversations": [item]}
    elif current_conversation is not None:
        current_conversation["conversations"].append(item)

# Add the last conversation
if current_conversation is not None:
    output_data.append(current_conversation)

# Save output data to a new JSON file
output_file = endfile

with open(output_file, "w") as file:
    json.dump(output_data, file, indent=2)

print(f"Converted conversations saved to {output_file}")


In [ ]:
###clean function


def delete_files(file_list):
    for file_name in file_list:
        try:
            os.remove(file_name)
            print(f"Deleted: {file_name}")
        except OSError as e:
            print(f"Error deleting {file_name}: {e}")

# List of files to delete


delete_files(filess)

